In [ ]:
import mne
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt

# --- Set up Project Root and Paths ---
# Assuming this notebook is in 'notebooks/', we go up one level to 'NTUT25_SOFTWARE'
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..', '..')) 

# Add 'src' directory to Python path to import config
import sys
sys.path.append(os.path.join(PROJECT_ROOT))

from src.eeg import config

# --- Define Paths ---
PROCESSED_EEG_DIR = os.path.join(config.PROCESSED_EEG_DATA_DIR, 'aggregated')
AGGREGATED_FNAME = os.path.join(PROCESSED_EEG_DIR, config.AGGREGATED_FNAME)

# --- Load the aggregated data ---
print(f"Loading aggregated data from: {AGGREGATED_FNAME}")

try:
    with open(AGGREGATED_FNAME, 'rb') as f:
        matched_data = pickle.load(f)
    print("Data loaded successfully.")
except FileNotFoundError:
    print(f"Error: Aggregated data file not found at {AGGREGATED_FNAME}.")
    matched_data = None

ModuleNotFoundError: No module named 'src'

In [ ]:
# To use MNE's plotting tools, we need to convert our NumPy arrays back to MNE objects.
# We'll use the info from a processed .fif file to get channel names and montage info.

# --- Load a reference Info object from a processed file ---
# Pick one of the processed .fif files created in the previous step
REF_FNAME_BASE = "804_1_PD_REST_processed"
REF_PROCESSED_FNAME = os.path.join(config.PROCESSED_EEG_DATA_DIR, f"{REF_FNAME_BASE}{config.RAW_FNAME_SUFFIX}")

try:
    epochs_ref = mne.read_epochs(REF_PROCESSED_FNAME, preload=False, verbose=False)
    info_ref = epochs_ref.info
    print(f"Loaded reference Info from {REF_PROCESSED_FNAME}.")
except FileNotFoundError:
    print(f"Error: Reference file not found at {REF_PROCESSED_FNAME}. Cannot reconstruct MNE objects.")
    info_ref = None

# --- Reconstruct Evoked Objects for each condition and group ---
if matched_data and info_ref:
    # matched_data['Eyes_Open'] is a list of tuples: [(PD_ON_data, PD_OFF_data, CTL_data), ...]
    # We need to average across subjects for each condition/group.
    
    # Concatenate data across subjects for each group
    pd_on_open_data = np.concatenate([d[0] for d in matched_data['Eyes_Open']], axis=0)
    pd_off_open_data = np.concatenate([d[1] for d in matched_data['Eyes_Open']], axis=0)
    ctl_open_data = np.concatenate([d[2] for d in matched_data['Eyes_Open']], axis=0)

    pd_on_closed_data = np.concatenate([d[0] for d in matched_data['Eyes_Closed']], axis=0)
    pd_off_closed_data = np.concatenate([d[1] for d in matched_data['Eyes_Closed']], axis=0)
    ctl_closed_data = np.concatenate([d[2] for d in matched_data['Eyes_Closed']], axis=0)
    
    # Create Evoked objects (which are just averages)
    info_ref_100hz = info_ref.copy()
    info_ref_100hz['sfreq'] = 100 # Match the final sampling rate
    
    pd_on_open_evoked = mne.EvokedArray(np.mean(pd_on_open_data, axis=0), info_ref_100hz, tmin=0)
    pd_off_open_evoked = mne.EvokedArray(np.mean(pd_off_open_data, axis=0), info_ref_100hz, tmin=0)
    ctl_open_evoked = mne.EvokedArray(np.mean(ctl_open_data, axis=0), info_ref_100hz, tmin=0)
    
    pd_on_closed_evoked = mne.EvokedArray(np.mean(pd_on_closed_data, axis=0), info_ref_100hz, tmin=0)
    pd_off_closed_evoked = mne.EvokedArray(np.mean(pd_off_closed_data, axis=0), info_ref_100hz, tmin=0)
    ctl_closed_evoked = mne.EvokedArray(np.mean(ctl_closed_closed_data, axis=0), info_ref_100hz, tmin=0)

In [ ]:
# MNE provides a convenient way to visualize electrode locations on a head model
if info_ref:
    fig = mne.viz.plot_sensors(info_ref, kind='head')
    fig.set_size_inches(6, 6)
    plt.show()

    # To see the names more clearly
    fig = mne.viz.plot_sensors(info_ref, kind='topomap', show_names=True, ch_type='eeg')
    fig.set_size_inches(8, 8)
    plt.show()

In [ ]:
if pd_on_open_evoked:
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6), sharey=True)
    
    # Plot Eyes-Open ERPs for all groups
    mne.viz.plot_compare_evokeds(
        dict(PD_ON=pd_on_open_evoked, PD_OFF=pd_off_open_evoked, CTL=ctl_open_evoked),
        picks=['Fz', 'Cz', 'Pz'], # Pick some key EEG channels
        title='Eyes-Open ERP',
        axes=axes[0],
        show=False
    )
    axes[0].legend(loc='upper right')
    
    # Plot Eyes-Closed ERPs for all groups
    mne.viz.plot_compare_evokeds(
        dict(PD_ON=pd_on_closed_evoked, PD_OFF=pd_off_closed_evoked, CTL=ctl_closed_evoked),
        picks=['Fz', 'Cz', 'Pz'],
        title='Eyes-Closed ERP',
        axes=axes[1],
        show=False
    )
    axes[1].legend(loc='upper right')
    plt.tight_layout()
    plt.show()

In [ ]:
if pd_on_open_evoked:
    # A time window for a potential alpha rhythm peak (e.g., 0.5-2 seconds)
    alpha_window = (0.5, 2.0)

    # Plot topomap for Eyes-Closed (where alpha is prominent)
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))
    
    mne.viz.plot_topomap(pd_on_closed_evoked.data[:, pd_on_closed_evoked.time_as_index(alpha_window)[0]],
                         pd_on_closed_evoked.info,
                         axes=axes[0],
                         show=False,
                         vlim=(-4, 4), # Set consistent color limits for comparison
                         title='PD ON Closed')

    mne.viz.plot_topomap(pd_off_closed_evoked.data[:, pd_off_closed_evoked.time_as_index(alpha_window)[0]],
                         pd_off_closed_evoked.info,
                         axes=axes[1],
                         show=False,
                         vlim=(-4, 4),
                         title='PD OFF Closed')
                         
    mne.viz.plot_topomap(ctl_closed_evoked.data[:, ctl_closed_evoked.time_as_index(alpha_window)[0]],
                         ctl_closed_evoked.info,
                         axes=axes[2],
                         show=False,
                         vlim=(-4, 4),
                         title='CTL Closed')

    plt.tight_layout()
    plt.show()

In [ ]:
# The SOP doesn't mention TF for REST, but it was in your Oddball analysis.
# Let's show how you would do it on the epochs.

if matched_data and info_ref:
    # We need to take the raw epoched data for a group, not the average.
    epochs_ctl_open = matched_data['Eyes_Open'][0][2] # First pair, CTL data
    
    # The data in matched_data is already a NumPy array.
    # We need to reconstruct an MNE Epochs object to run TF on it.
    epochs_ctl_open_mne = mne.EpochsArray(epochs_ctl_open, info_ref_100hz, tmin=0, verbose=False)
    
    # Frequencies to analyze (e.g., 4 to 30 Hz for Alpha, Beta bands)
    freqs = np.arange(4, 30, 1)
    
    # Compute Morlet wavelet time-frequency representation
    power = mne.time_frequency.tfr_morlet(epochs_ctl_open_mne, freqs=freqs, 
                                          n_cycles=freqs/2, return_itc=False, verbose=False)
    
    # Plot the power (averaged across epochs)
    fig = power.plot_topo(title='Time-Frequency Power (CTL Eyes-Open)', vlim=(-1, 1),
                          colorbar=True, show=False)
    plt.show()